In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from random import randint
from sklearn.metrics import accuracy_score
from sklearn import model_selection, metrics
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation

In [2]:
data = pd.read_csv('dataset_conlang.csv')

### Предобработка данных

In [3]:
data.head()

,conlang_name,conlang_type,conlang_status,conlang_phonems,name,language
0,"['Tirina [TNA]', 'Sound samples in Tirina', 'L...",['A priori'],['Functional'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",['alynnidalar'],['English']
1,"['&apos;DraUz [DRZ]', 'Phonology', 'Orthography']",['Artistic Language (Artlang)'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",['Dolfyn'],['English']
2,"['&apos;Nga [RNI]', 'Phonology', 'Orthography']",['A priori'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['Orikrin'],['French (Fran\xc3\xa7ais)']
3,"['(Desert) Elvish [EL2]', 'Language family rel...",['A priori'],['Functional'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",['Cestion'],['English']
4,"['(no name yet) [A12]', 'Language family relat...",['A priori'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['Kougo'],['English']


In [198]:
conlangs = data[data.conlang_type == "['Artistic Language (Artlang)']"].append(
    data[data.conlang_type == "['A priori']"])
conlangs.language.value_counts()

['English']                                                                                                                    1536
['German (Deutsch)']                                                                                                             66
['French (Fran\xc3\xa7ais)']                                                                                                     63
['Swedish (Svenska)']                                                                                                            59
['Portuguese (Portugu\xc3\xaas)']                                                                                                46
['Dutch (Nederlands)']                                                                                                           39
['Spanish (Espa\xc3\xb1ol)']                                                                                                     39
['Italian (Italiano)']                                                      

In [199]:
conlangs.index = range(0,len(conlangs))

In [200]:
interesting_language = conlangs.language.value_counts().index[0:8]

In [201]:
main_dataset = list()
indexes = list()
for i in range(0,len(interesting_language)):
    phonems = conlangs[conlangs.language == interesting_language[i]].conlang_phonems
    index = conlangs[conlangs.language == interesting_language[i]].index
    for j, phonem in enumerate(phonems):
        main_dataset.append([phonem, i])
        indexes.append(index[j])
for i in range(0,len(main_dataset)):
    main_dataset[i][0] = map(float, main_dataset[i][0][1:-1].split(', '))

In [202]:
main_dataset = main_dataset[1490:]
indexes = indexes[1490:]

In [203]:
len(main_dataset)

385

In [205]:
X = pd.DataFrame(data = list(np.asarray(main_dataset).T[0]))
Y = pd.DataFrame(data = list(np.asarray(main_dataset).T[1]))
X.index = indexes
Y.index = indexes

### Классификатор из коробки

In [206]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [207]:
model = RandomForestClassifier(n_jobs = -1, random_state = 0)
model.fit(X_train, y_train)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [208]:
model.score(X_train, y_train)

0.96282527881040891

In [209]:
model.score(X_test, y_test)

0.26724137931034481

In [14]:
#Работает лучше, чем случайность, ибо:
100.0 / 8

12.5

In [15]:
model.predict(X_test)

array([1, 3, 4, 6, 4, 2, 0, 2, 1, 5, 7, 7, 1, 4, 3, 3, 3, 1, 0, 5, 3, 1, 0,
       1, 2, 3, 2, 3, 1, 0, 2, 3, 0, 3, 3, 7, 3, 3, 6, 2, 3, 1, 1, 0, 1, 4,
       5, 1, 0, 4, 3, 1, 1, 2, 3, 1, 3, 1, 4, 3, 0, 4, 0, 5, 3, 1, 3, 2, 0,
       3, 3, 0, 3, 3, 3, 3, 1, 1, 1, 1, 6, 0, 3, 0, 5, 4, 2, 0, 3, 4, 1, 1,
       4, 1, 4, 5, 2, 4, 2, 1, 2, 1, 3, 2, 1, 1, 3, 3, 4, 2, 1, 0, 2, 2, 2,
       1])

In [16]:
print y_test

[4, 4, 6, 6, 1, 7, 0, 2, 7, 4, 4, 2, 3, 4, 1, 3, 2, 7, 7, 4, 0, 0, 0, 1, 0, 1, 2, 6, 0, 5, 1, 0, 6, 1, 2, 6, 3, 1, 6, 5, 1, 2, 2, 2, 2, 0, 0, 1, 1, 1, 0, 1, 5, 3, 2, 5, 3, 1, 0, 3, 0, 4, 1, 4, 3, 3, 6, 2, 7, 0, 0, 2, 1, 2, 1, 3, 1, 2, 6, 3, 1, 3, 6, 3, 3, 7, 6, 3, 1, 5, 5, 0, 1, 5, 0, 0, 2, 6, 6, 1, 2, 4, 7, 5, 1, 1, 6, 3, 6, 2, 1, 0, 2, 0, 3, 6]


### Кросс валидация и подбор параметров

In [17]:
forest = RandomForestClassifier(n_jobs = -1, random_state = 0)
parameter_grid = {
    'n_estimators' : [500, 1000, 1500, 2000],
    'max_depth' : [5, 7, 9, 13, 17],
    'min_samples_split' : [2, 4, 8, 10],
    'min_samples_leaf' : [1, 2],
}

In [20]:
%%time
grid_search = GridSearchCV(forest, parameter_grid, scoring='accuracy', 
                    cv =  cross_validation.StratifiedShuffleSplit(Y, n_iter = 5, test_size = 0.3, random_state=0))
grid_search.fit(X, Y)
best_params = grid_search.best_params_
print best_params

{'min_samples_split': 4, 'n_estimators': 1500, 'max_depth': 13, 'min_samples_leaf': 1}
CPU times: user 1h 24min 39s, sys: 11min 40s, total: 1h 36min 20s
Wall time: 1h 29min 22s


In [210]:
model = RandomForestClassifier(n_jobs = -1, random_state = 0, **best_params)
model.fit(X_train, y_train)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            n_estimators=1500, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [211]:
model.score(X_train, y_train)

0.97769516728624539

In [212]:
model.score(X_test, y_test)

0.32758620689655171

In [213]:
cvs = cross_val_score(model, X, Y[0], scoring="accuracy", cv = 10)
print round(np.mean(cvs), 2)

0.32


In [214]:
predict = model.predict(X_test)

In [218]:
wrong_indexes = list()
wrong_answers = list()
for i, j in enumerate(y_test.index):
    if y_test[0][j] != predict[i]:
        wrong_indexes.append(j)
        wrong_answers.append(predict[i])

In [223]:
for i in range(0,len(wrong_answers)):
    print interesting_language[wrong_answers[i]]
    print conlangs.loc[wrong_indexes[i]]
    print ''

['Swedish (Svenska)']
conlang_name       ['Rhwoqh Language [RWQ]', 'Phonology', 'Orthog...
conlang_type                                            ['A priori']
conlang_status                                               ['New']
conlang_phonems    [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
name                                              ['Gustavo Varela']
language                           ['Portuguese (Portugu\xc3\xaas)']
Name: 1835, dtype: object

['Swedish (Svenska)']
conlang_name       ['Prrt&apos;ek&apos; [RRR]', 'Phonology', 'Ort...
conlang_type                                            ['A priori']
conlang_status                                       ['Progressing']
conlang_phonems    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
name                                                      ['Kjades']
language                                ['Spanish (Espa\xc3\xb1ol)']
Name: 1779, dtype: object

['Portuguese (Portugu\xc3\xaas)']
conlang_name        ['Ro&apos;an [ROA]',

In [217]:
conlangs.loc[wrong_indexes]

,conlang_name,conlang_type,conlang_status,conlang_phonems,name,language
1835,"['Rhwoqh Language [RWQ]', 'Phonology', 'Orthog...",['A priori'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['Gustavo Varela'],['Portuguese (Portugu\xc3\xaas)']
1779,"['Prrt&apos;ek&apos; [RRR]', 'Phonology', 'Ort...",['A priori'],['Progressing'],"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",['Kjades'],['Spanish (Espa\xc3\xb1ol)']
548,"['Ro&apos;an [ROA]', 'Phonology', 'Orthography']",['Artistic Language (Artlang)'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['xroooox'],['Spanish (Espa\xc3\xb1ol)']
364,"['Lyserian (Eastern) [LYE]', 'Language family ...",['Artistic Language (Artlang)'],['New'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",['trima\xc3\xa9tres'],['German (Deutsch)']
773,"['Xingwa [XGW]', 'Sound samples in Xingwa', 'P...",['Artistic Language (Artlang)'],['Progressing'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['Dari Benetka'],['Italian (Italiano)']
1016,"['Ctann [KTQ]', 'Phonology', 'Orthography']",['A priori'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['Dari Benetka'],['Italian (Italiano)']
560,"['Saskia [SKI]', 'Phonology', 'Orthography']",['Artistic Language (Artlang)'],['Progressing'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['Julia Rabelo da Cunha'],['Portuguese (Portugu\xc3\xaas)']
215,"['Ghethtugian [GFS]', 'Facts', 'Phonology', 'O...",['Artistic Language (Artlang)'],[],"[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['Orikrin'],['French (Fran\xc3\xa7ais)']
2,"['??? [NUH]', 'Phonology', 'Orthography']",['Artistic Language (Artlang)'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['GohtCheee'],['Swedish (Svenska)']
1268,"['Ivu [IVU]', 'Phonology', 'Orthography']",['A priori'],['New'],"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",['TuxCrafting'],['French (Fran\xc3\xa7ais)']
